### `Ali Almalki`

In this tutorial, I am going to develop an end-to-end domain-specific intelligent chatbot solution using deep learning with Keras.

In [27]:
# Import all required libraries  
import numpy as np 
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle
import colorama 
colorama.init()
from colorama import Fore, Style, Back

### Load dataset

In [5]:
with open("intents.json") as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data["intents"]:
    for pattern in intent["patterns"]:
        training_sentences.append(pattern)
        training_labels.append(intent["tag"])
    responses.append(intent["responses"])
    
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
        
num_classes = len(labels)
print(f"- Number of classes are: {num_classes} classes")

- Number of classes are: 8 classes


`training_sentences`: holds all the training data (which are the sample messages in each intent category) 

`training_labels`: holds all the target labels correspond to each training data.


In [7]:
# Check out our training sentences
training_sentences

['Hi',
 'Hey',
 'Is anyone there?',
 'Hello',
 'Hay',
 'Bye',
 'See you later',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Who are you?',
 'What are you?',
 'Who you are?',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'Could you help me?',
 'give me a hand please',
 'Can you help?',
 'What can you do for me?',
 'I need a support',
 'I need a help',
 'support me please',
 'I need to create a new account',
 'how to open a new account',
 'I want to create an account',
 'can you create an account for me',
 'how to open a new account',
 'have a complaint',
 'I want to raise a complaint',
 'there is a complaint about a service']

In [8]:
# Check out our training labels 
training_labels

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'about',
 'about',
 'about',
 'name',
 'name',
 'name',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help',
 'createaccount',
 'createaccount',
 'createaccount',
 'createaccount',
 'createaccount',
 'complaint',
 'complaint',
 'complaint']

### Encode training labels 

In [9]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

print(training_labels)

[4 4 4 4 4 3 3 3 7 7 7 7 0 0 0 6 6 6 5 5 5 5 5 5 5 2 2 2 2 2 1 1 1]


In [14]:
# Check out a sample from our training label
training_labels[0]

4

### Vectorize our training sentences

In [15]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>" # “out of token” to deal with out of vocabulary words(tokens) at inference time.

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating="post", maxlen=max_len) # The “pad_sequences” method is used to make all the training text sequences into the same size.

### Modelling Part

In [21]:
# Set a seed 
tf.random.set_seed(42)

# Create the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(num_classes, activation="softmax")) # Number of classes = 8

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="adam", 
              metrics=["accuracy"])

# Fit the model 
history = model.fit(padded_sequences, np.array(training_labels), epochs=600)


Epoch 1/600
2/2 [==============================] - 0s 7ms/step - loss: 2.0801 - accuracy: 0.0606
Epoch 2/600
2/2 [==============================] - 0s 3ms/step - loss: 2.0784 - accuracy: 0.0909
Epoch 3/600
2/2 [==============================] - 0s 6ms/step - loss: 2.0777 - accuracy: 0.0909
Epoch 4/600
2/2 [==============================] - 0s 4ms/step - loss: 2.0768 - accuracy: 0.0909
Epoch 5/600
2/2 [==============================] - 0s 4ms/step - loss: 2.0762 - accuracy: 0.0909
Epoch 6/600
2/2 [==============================] - 0s 4ms/step - loss: 2.0758 - accuracy: 0.0909
Epoch 7/600
2/2 [==============================] - 0s 2ms/step - loss: 2.0752 - accuracy: 0.0909
Epoch 8/600
2/2 [==============================] - 0s 7ms/step - loss: 2.0748 - accuracy: 0.0909
Epoch 9/600
2/2 [==============================] - 0s 4ms/step - loss: 2.0746 - accuracy: 0.0909
Epoch 10/600
2/2 [==============================] - 0s 4ms/step - loss: 2.0741 - accuracy: 0.0909
Epoch 11/600
2/2 [===========

In [22]:
# Check out the model architecture 
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_3   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_9 (Dense)             (None, 16)                272       
                                                                 
 dense_10 (Dense)            (None, 16)                272       
                                                                 
 dense_11 (Dense)            (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

### Save Model 

In [24]:
# Save the trained model
model.save("chat_model")

# Save the fitted tokenizer
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# Save the fitted label encoder
with open("label_encoder.pickle", "wb") as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

### Model Prediction

In [28]:
def chat():
    # load trained model
    model = keras.models.load_model("chat_model")

    # load tokenizer object
    with open("tokenizer.pickle", "rb") as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open("label_encoder.pickle", "rb") as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating="post", maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data["intents"]:
            if i["tag"] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i["responses"]))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)

chat()

Start messaging with the bot (type quit to stop)!
User: hello
ChatBot: Hello
User: How is your day?
ChatBot: Hi there
User: My name is Ali, and you?
ChatBot: You can call me Joana.
User: Nice to meet you Joana
ChatBot: Hi there
User: Gotta go now. Have a good day!
ChatBot: Please mention your complaint, we will reach you and sorry for any inconvenience caused
User: Goodbye!
ChatBot: Hi there
User: bye
ChatBot: Hello
User: goodbye
ChatBot: Hi
User: quit


### References: 

* [How To Build Your Own Chatbot Using Deep Learning](https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281)

* [Dataset (intents.json](https://drive.google.com/file/d/1gjNEWb06kA6mRFXmhYt5JX2zXKx7BYu2/view?usp=sharing))